In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

# Read the file
os.chdir("C://specdata/fraud_kaggle")
cc = pd.read_csv("creditcard.csv")

In [2]:
#Data check

cc.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
#I observed an conflict in the name 'class'. Therefore, I have changed the name from class to category

cc= cc.rename(columns={'Class': 'Category'})

In [4]:
# For convinience, divide the dataframe cc based on two labels. 

nor_obs = cc.loc[cc.Category==0]    #Data frame with normal observation
ano_obs = cc.loc[cc.Category==1]    #Data frame with anomalous observation

In [5]:
# The given dataframe 'cc' is divided into three sets 
# Training set: train_features
# Test observations/features: X_test
# Test labels: Y_test

In [6]:
# Once class SVM is trained with the observations of only one class. In this case, the algorithm is trained with 
# first 200,000 observation of normal transactions. The remaining observation is merged with the anomalous observation 
# to create a test set. 

train_feature = nor_obs.loc[0:200000, :]
train_feature = train_feature.drop('Category', 1)

In [7]:
# Creatng test observations/features

X_test_1 = nor_obs.loc[200000:, :].drop('Category',1)
X_test_2 = ano_obs.drop('Category',1)
X_test = X_test_1.append(X_test_2)

In [8]:
# The remain data set is (after 200,000 observations) are appended with anomalous observations

Y_1 = nor_obs.loc[200000:, 'Category']
Y_2 = ano_obs['Category']

Y_test= Y_1.append(Y_2)

#Y_test is used to evaluste the model

In [9]:
# Setting the hyperparameters for Once Class SVM

oneclass = svm.OneClassSVM(kernel='linear', gamma=0.001, nu=0.95)

# I have used various combination of hyperparameters like linear, rbf, poly, gamma- 0.001, 0.0001, nu- 0.25, 0.5, 0.75, 0.95
# This combination gave me the most satisfactory results.

In [10]:
# Training the algorithm with the features. 
# This stage is very time consuming processes. In my laptop it took more than an hour to train for 200,000 observations. 
# For rbf, the time taken is even more.

oneclass.fit(train_feature)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='linear',
      max_iter=-1, nu=0.95, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [11]:
# Test the algorithm on the test set

fraud_pred = oneclass.predict(X_test)

In [12]:
# Check the number of outliers predicted by the algorithm

unique, counts = np.unique(fraud_pred, return_counts=True)
print (np.asarray((unique, counts)).T)

[[ -1.00000000e+00   3.71000000e+02]
 [  1.00000000e+00   8.48210000e+04]]


In [13]:
#Convert Y-test and fraud_pred to dataframe for ease of operation

Y_test= Y_test.to_frame()
Y_test=Y_test.reset_index()
fraud_pred = pd.DataFrame(fraud_pred)
fraud_pred= fraud_pred.rename(columns={0: 'prediction'})


In [14]:
##Performance check of the model

TP = FN = FP = TN = 0
for j in range(len(Y_test)):
    if Y_test['Category'][j]== 0 and fraud_pred['prediction'][j] == 1:
        TP = TP+1
    elif Y_test['Category'][j]== 0 and fraud_pred['prediction'][j] == -1:
        FN = FN+1
    elif Y_test['Category'][j]== 1 and fraud_pred['prediction'][j] == 1:
        FP = FP+1
    else:
        TN = TN +1
print (TP,  FN,  FP,  TN)



84700 0 121 371


In [15]:
# Performance Matrix

accuracy = (TP+TN)/(TP+FN+FP+TN)
print (accuracy)
sensitivity = TP/(TP+FN)
print (sensitivity)
specificity = TN/(TN+FP)
print (specificity)

0.9985796788430839
1.0
0.7540650406504065


Following results were obtained 
accuracy= 99.9%
sensitivity = 100%
specificity = 75%
Once class SVM has shown a very promising performance for this dataset with near 90% detection of anomaly and very few false alarm. This can be a starting point for fine tuning the algorthm to improve the specificity, keeping other things constant. Tuning the hyperparameters are very time consuming process and the Kaggle kernal stops after some time. Therefore, O couldnt run the code. I have just shown my codes in the cell. I am sure this code will run because i have ran it in my Jupyter note book. I have also isolation forest in my previous kernal. Once class SVM seems to outperform isolation forest in this case. 
